In [0]:
import tensorflow as tf
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [0]:
import os
import math
import pandas as pd
from google.colab import drive
drive.mount('/content/Drive')

Drive already mounted at /content/Drive; to attempt to forcibly remount, call drive.mount("/content/Drive", force_remount=True).


In [0]:
cd "/content/Drive/My Drive/Spring 2019/ML/sample for project"

/content/Drive/My Drive/Spring 2019/ML/sample for project


In [0]:
df=pd.read_csv('vector_form.csv')

In [0]:
df.shape

(11429, 13763)

In [0]:
col= ['id_y', 'postMedia', 'postText', 'postTimestamp', 'targetCaptions', 'targetDescription',	'targetKeywords',	'targetParagraphs',	'targetTitle',	'truthClass',	'truthJudgments',	'truthMean',	'truthMedian',	'truthMode']
len(col)

14

In [0]:
image_col = ['person_y',
 'bicycle',
 'car_y',
 'motorcycle',
 'airplane',
 'bus',
 'train_y',
 'truck_y',
 'boat_y',
 'traffic light',
 'fire hydrant',
 'stop sign',
 'parking meter',
 'bench_y',
 'bird_y',
 'cat_y',
 'dog_y',
 'horse',
 'sheep_y',
 'cow_y',
 'elephant',
 'bear_y',
 'zebra_y',
 'giraffe',
 'backpack_y',
 'umbrella_y',
 'handbag_y',
 'tie_y',
 'suitcase',
 'frisbee',
 'skis',
 'snowboard',
 'sports ball',
 'kite',
 'baseball bat',
 'baseball glove',
 'skateboard_y',
 'surfboard',
 'tennis racket',
 'bottle',
 'wine glass',
 'cup_y',
 'fork_y',
 'knife_y',
 'spoon',
 'bowl_y',
 'banana_y',
 'apple',
 'sandwich_y',
 'orange',
 'broccoli',
 'carrot',
 'hot dog',
 'pizza_y',
 'donut_y',
 'cake_y',
 'chair_y',
 'couch',
 'potted plant',
 'bed_y',
 'dining table',
 'toilet_y',
 'tv_y',
 'laptop_y',
 'mouse',
 'remote',
 'keyboard_y',
 'cell phone',
 'microwave',
 'oven_y',
 'toaster_y',
 'sink_y',
 'refrigerator',
 'book_y',
 'clock_y',
 'vase',
 'scissors',
 'teddy bear',
 'hair drier',
 'toothbrush']

len(image_col)

80

In [0]:
# normalizing the image representation
# clipping to 2 instead of 1 to reserve the multiple objects in image

for col_name in image_col:
  for i in range(len(df)):
    if df[col_name][i] >2:
      df.set_value(i, col_name, 2)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  """


In [0]:
df = df.drop(col, axis = 1)

In [0]:
df.shape

(11429, 13749)

In [0]:
truth = pd.DataFrame(df['truthClassInt'])

In [0]:
df=df.drop('truthClassInt',axis = 1)

In [0]:
df.shape, truth.shape

((11429, 13748), (11429, 1))

Split training/validating data and test data (train/validate split follows below)

In [0]:
from sklearn.model_selection import train_test_split
df, X_test, truth, Y_test = train_test_split(df, truth, test_size = 0.20)

In [0]:
df.shape, X_test.shape, truth.shape, Y_test.shape

((9143, 13748), (2286, 13748), (9143, 1), (2286, 1))

Split the df and truth to test and validate for training

In [0]:
import numpy as np
import keras
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Convolution1D, Dropout, MaxPooling1D
from keras.optimizers import SGD
from keras.utils import np_utils

*1. change the dataframe to np array*

In [0]:
df_array = np.array(df.iloc[:,:].values)
df_array.shape

(9143, 13748)

In [0]:
truth_array = np.array(truth)
truth_array.shape

(9143, 1)

In [0]:
df_array[:3]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [0]:
#standardize train feature
scaler = StandardScaler().fit(df_array)
scaled_train = scaler.transform(df_array)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [0]:
scaled_train[:3]

array([[-0.01811705, -0.01045874, -0.01045874, ..., -0.0875755 ,
        -0.03709704, -0.08427205],
       [-0.01811705, -0.01045874, -0.01045874, ..., -0.0875755 ,
        -0.03709704, -0.08427205],
       [-0.01811705, -0.01045874, -0.01045874, ..., -0.0875755 ,
        -0.03709704, -0.08427205]])

In [0]:
scaled_train.shape

(9143, 13748)

In [0]:
sss = StratifiedShuffleSplit(test_size=0.1, random_state=10)
for train_index, valid_index in sss.split(scaled_train, truth_array):
    X_train, X_valid = scaled_train[train_index], scaled_train[valid_index]
    y_train, y_valid = truth_array[train_index], truth_array[valid_index]

In [0]:
X_train.shape, y_train.shape, X_valid.shape, y_valid.shape

((8228, 13748), (8228, 1), (915, 13748), (915, 1))

In [0]:
nrows, ncols = X_train.shape
nrows, ncols

(8228, 13748)

In [0]:
valid_rows = len(X_valid)
valid_rows

915

In [0]:
X_train = X_train.reshape(nrows,ncols, 1)
X_train[:3]

array([[[-0.01811705],
        [-0.01045874],
        [-0.01045874],
        ...,
        [-0.0875755 ],
        [-0.03709704],
        [-0.08427205]],

       [[-0.01811705],
        [-0.01045874],
        [-0.01045874],
        ...,
        [-0.0875755 ],
        [-0.03709704],
        [-0.08427205]],

       [[-0.01811705],
        [-0.01045874],
        [-0.01045874],
        ...,
        [-0.0875755 ],
        [-0.03709704],
        [-0.08427205]]])

In [0]:
X_train.shape

(8228, 13748, 1)

In [0]:
X_valid = X_valid.reshape(valid_rows, ncols, 1)
X_valid.shape

(915, 13748, 1)

In [0]:
#model with Cov1d layer
model = Sequential()
model.add(Convolution1D(filters=100, kernel_size=10,activation='relu',input_shape =(ncols,1) ))
model.add(Convolution1D(filters=100, kernel_size=10, activation='relu'))
model.add(MaxPooling1D(3))
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(2, activation='softmax'))
print(model.summary())



Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 13739, 100)        1100      
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 13730, 100)        100100    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 4576, 100)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 457600)            0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 457600)            0         
_________________________________________________________________
dense_1 (Dense)      

In [0]:
y_train = np_utils.to_categorical(y_train, 2)
y_train.shape

(8228, 2)

In [0]:
#skip
y_train = y_train.reshape(nrows, 2, 1)
y_train.shape

(8228, 2, 1)

In [0]:
y_train[:10]

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.]], dtype=float32)

In [0]:
y_valid = np_utils.to_categorical(y_valid, 2)
y_valid.shape

(915, 2)

In [0]:
#skip
y_valid = y_valid.reshape(valid_rows, 2, 1)
y_valid.shape

(915, 2, 1)

In [0]:
sgd = SGD(lr=0.01, nesterov=True, decay=1e-6, momentum=0.9)
model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])

In [0]:
nb_epoch = 10
model.fit(X_train, y_train, epochs=nb_epoch, validation_data=(X_valid, y_valid), batch_size=16)

Instructions for updating:
Use tf.cast instead.
Train on 8228 samples, validate on 915 samples
Epoch 1/10
8228/8228 [==============================] - 45s 5ms/step - loss: 0.5197 - acc: 0.7626 - val_loss: 0.5145 - val_acc: 0.7563
Epoch 2/10
8228/8228 [==============================] - 40s 5ms/step - loss: 0.4095 - acc: 0.8162 - val_loss: 0.5666 - val_acc: 0.7749
Epoch 3/10
8228/8228 [==============================] - 40s 5ms/step - loss: 0.3515 - acc: 0.8470 - val_loss: 0.7036 - val_acc: 0.7803
Epoch 4/10
8228/8228 [==============================] - 40s 5ms/step - loss: 0.2972 - acc: 0.8745 - val_loss: 0.5353 - val_acc: 0.7596
Epoch 5/10
8228/8228 [==============================] - 40s 5ms/step - loss: 0.2365 - acc: 0.9030 - val_loss: 0.7992 - val_acc: 0.7585
Epoch 6/10
8228/8228 [==============================] - 40s 5ms/step - loss: 0.2074 - acc: 0.9171 - val_loss: 0.6679 - val_acc: 0.7749
Epoch 7/10
8228/8228 [==============================] - 40s 5ms/step - loss: 0.1835 - acc: 0.92

In [0]:
print(X_test.shape)
X_test = np.array(X_test)
print(len(X_test))

X_test = X_test.reshape(len(X_test), ncols, 1)
X_test.shape

(2286, 13748)
2286


(2286, 13748, 1)

In [0]:
y_valid.shape, Y_test.shape

((915, 2), (2286, 1))

In [0]:
Y_test = np.array(Y_test)
Y_test = np_utils.to_categorical(Y_test, 2)
Y_test.shape

(2286, 2)

In [0]:
result = model.predict(X_test)

In [0]:
result

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [0]:
from sklearn.metrics import accuracy_score

In [0]:
accuracy_score(Y_test, result)

0.7773403324584427